# Generation ChemBERT (Chemical Language Model Embedding) Condensation_reactions

Found this dataset from a recent publication by a D.M. Makarov et al. (Journal of Computational Science 74 (2023) 102173) (https://doi.org/10.1016/j.jocs.2023.102173)

"""We considered pyrrole or dipyrromethane condensation reactions with various aldehydes, resulting in the production of boron(III) dipyrromethene or BODIPY (681 records). These reactions were retrieved from articles (see “Dataset reactions” and Scheme S1). Addi tionally, we used the reactions of the production of dipyrromethane (111 records) and porphyrins (457 records). All condensation reactions for dipyrromethanes and 213 reactions for porphyrins with various al dehydes were obtained in our laboratory. The remaining 244 reactions for the porphyrins synthesis were obtained from articles (see “Dataset reactions”). Our experimental dataset is based on a study of pyrrole condensation processes with aldehydes, using catalytic amounts of organic acids to produce ms-aryl- and ß-alkyl-substituted dipyrro methanes. """

The objective of this notebook is to introduce a method for predicting yield

As a beginner, there may be numerous opportunities for improvement in this notebook. I was largely inspired by the work of D.M. Makarov et al and the STEPHEN LEE's notebook (BELKA: Molecule Representations for ML Tutorial) thanks to them.

## Data

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_row', 211)
pd.set_option('display.max_columns',211)

In [3]:
# Téléchargement du Dataset 
DataFrame = r"C:\Users\loris\Desktop\IA_chemistry\Dipyrromethanes_condensation_reactions-main\Condensation_reactions - Copie.xlsx"

df = pd.read_excel(DataFrame)

df= df.copy()

print(df.shape)
df.head()

(1249, 4)


,SMILES,yield,Temperature,Ind
0,CC(C)(C)c1ccc(OCCCCOc2ccc(C(C)(C)C)cc2C=O)c(C=...,0.05,20,78
1,O=Cc1ccccc1OCCOC(=O)C#CC(=O)OCCOc1ccccc1C=O.c1...,1.00,40,909
2,c1cc[nH]c1.Cc1cc(C)c(C=O)c(C)c1.COC(=O)c1ccc(C...,2.00,20,747
3,CC(C)(C)c1ccc(OCCCCCOc2ccc(C(C)(C)C)cc2C=O)c(C...,2.10,20,77
4,Cc1ccc(-c2ccccc2C=O)cc1.Fc1c(F)c(F)c(C(c2ccc[n...,2.50,25,266


In [4]:
df["SMILES"][0]

'CC(C)(C)c1ccc(OCCCCOc2ccc(C(C)(C)C)cc2C=O)c(C=O)c1.CC(C)(C)c1ccc(OCCCCOc2ccc(C(C)(C)C)cc2C(c2ccc[nH]2)c2ccc[nH]2)c(C(c2ccc[nH]2)c2ccc[nH]2)c1.CCC(=O)O.CC(=O)O.O=[N+]([O-])c1ccccc1>>CC(C)(C)c1ccc2c(c1)-c1c3nc(c4c5ccc([nH]5)c(c5nc(c(c6ccc1[nH]6)-c1cc(C(C)(C)C)ccc1OCCCCOc1ccc(C(C)(C)C)cc1-4)C=C5)-c1cc(C(C)(C)C)ccc1OCCCCO2)C=C3.O.O'

## Missing values

In [5]:
# Recherche des valeurs manquantes
missing_values = df.isnull().sum()
missing_values

SMILES         0
yield          0
Temperature    0
Ind            0
dtype: int64

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

# Création de la figure et des axes pour les subplots
fig, axs = plt.subplots(2, 1, figsize=(10,8))

# Tracé de l'histogramme sur le premier subplot
sns.histplot(data=df, x='yield', bins=30, ax=axs[0])
axs[0].set_xlabel('Rendement chimique (%)')
axs[0].set_ylabel("Nombre d'occurences")
axs[0].set_title("Distribution des rendements chimiques")

# Tracé du boxplot sur le deuxième subplot
sns.boxplot(df["yield"], ax=axs[1])
axs[1].set_ylabel("Rendement chimique (%)")
axs[1].set_title("Distribution des rendements chimiques")

# Ajustement de l'espacement entre les subplots
plt.tight_layout()

# Affichage des graphiques 
plt.show()

ModuleNotFoundError: No module named 'seaborn'

## 🤖 Chemical Language Model Embeddings

Just as traditional language models use self-attention mechanisms to compute the representation of each language element (e.g. word in a sentence) to every other element, chemical language models use the same principle in which elements are some chemical unit (e.g. atoms) instead of words.

Below is example code to obtain learned transformer-based embeddings from two chemical language models:

ChemBERTa: adapted for chemical SMILES from the RoBERTa architecture, trained on a dataset of 77 million molecules

MoLFormer: another transformer-based model adapted for SMILES but trained on a larger dataset (1.1 billion molecules!)


References:

Chithrananda, S., Grand, G., & Ramsundar, B. (2020). ChemBERTa: Large-Scale Self-Supervised Pretraining for Molecular Property Prediction.
    
Chithrananda, S., Grand, G., & Ramsundar, B. (2022). ChemBERTa-2: Towards Chemical Foundation Models. 

Ross, J., Belgodere, B., Chenthamarakshan, V., et al. (2022). Large-scale chemical language representations capture molecular structure and properties. 
Nature Machine Intelligence, 4, 1256-1264. 

ChemBERTa on HuggingFace Model Repo

MolFormer on HuggingFace Model Repo

MolFormer GitHub repo

In [7]:
# for chemical language models
from transformers import AutoModel, AutoTokenizer
import torch

C:\Users\loris\anaconda3\envs\rxnfp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
smiles = df["SMILES"]

In [9]:
# load pre-trained ChemBERTa model checkpoint and tokenizer
cb_tokenizer = AutoTokenizer.from_pretrained('DeepChem/ChemBERTa-10M-MLM')
cb_model = AutoModel.from_pretrained('DeepChem/ChemBERTa-10M-MLM')
cb_model.eval()

# tokenize SMILES
cb_encoded_inputs = cb_tokenizer(list(smiles), padding=True, truncation=True, return_tensors="pt")

# calculate embeddings
with torch.no_grad():
    outputs = cb_model(**cb_encoded_inputs)

# extract pooled output
cb_embeddings = outputs.pooler_output

cb_embeddings_df = pd.DataFrame(cb_embeddings.numpy())
cb_embeddings_df.head()

Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MLM and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,...,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383
0,-0.162271,0.359961,-0.208631,0.212792,-0.184559,0.023665,0.031174,-0.056128,0.070213,0.246463,0.404678,0.222561,0.246697,0.569248,-0.286271,0.163123,-0.211469,-0.137589,0.333122,0.270672,-0.171357,-0.007442,0.026922,0.093674,0.563860,0.190829,-0.260796,0.335964,0.166725,-0.452981,-0.144069,0.137669,0.074681,-0.076224,0.409972,-0.091544,-0.143137,-0.533438,-0.177174,0.303646,-0.170809,0.113998,-0.180275,-0.298499,0.129155,-0.358678,-0.197952,-0.127823,0.095293,0.146926,0.200765,-0.178596,0.116211,-0.196614,0.198353,0.067136,0.164220,0.255172,-0.430976,-0.052471,0.122035,-0.120684,0.194026,0.059159,-0.242641,0.216599,-0.209428,-0.450314,-0.067776,0.322422,0.050678,-0.037782,0.081106,0.362837,-0.256037,-0.186787,-0.048138,-0.397506,-0.240361,-0.247374,0.147705,0.132116,0.320903,-0.256691,-0.021490,-0.365033,-0.155543,0.344853,0.302176,-0.139125,0.172635,-0.128772,0.321327,0.151302,-0.063909,-0.135832,0.332110,-0.003239,0.258752,0.012680,-0.174956,0.260750,0.035437,0.454084,0.084206,...,0.325775,0.087837,-0.047203,-0.375325,0.240884,0.096204,0.210107,0.152762,-0.285373,-0.281662,0.243171,0.335456,0.193422,-0.344888,-0.180593,0.024781,-0.214996,-0.305409,-0.109843,-0.137799,-0.499643,0.295993,0.223503,0.012261,-0.147910,0.103415,-0.091400,0.090393,0.178400,-0.128588,0.159457,0.131086,-0.192747,0.246661,-0.018331,-0.004980,0.066220,0.229715,0.022504,-0.275646,0.057845,0.330876,0.000617,-0.515452,0.235576,0.299304,0.124795,0.086530,0.011721,-0.124608,0.353909,0.176914,0.021789,-0.560504,-0.019877,0.027930,-0.026669,-0.124045,0.082781,-0.416438,0.100424,0.171194,-0.240402,-0.120383,-0.101247,-0.219663,-0.180439,0.171812,-0.378690,-0.122113,0.160866,0.131128,0.284670,0.330584,-0.425320,0.112695,0.506408,0.361750,-0.046845,0.007392,-0.070656,-0.199145,0.104572,-0.008363,-0.072674,-0.080588,-0.084459,0.144243,0.227459,-0.067456,-0.541151,0.081943,-0.373041,-0.345800,0.036262,0.095997,-0.096873,-0.063860,-0.114892,0.092941,-0.445563,-0.034386,-0.321318,-0.407256,0.045911
1,-0.164998,0.200312,-0.291708,0.385741,-0.113083,0.090308,-0.015352,0.234842,0.140149,0.200789,0.300456,0.268847,0.327487,0.552415,-0.058277,0.069845,-0.283775,0.059433,0.132053,0.206883,0.020623,-0.173855,0.225282,-0.017904,0.497691,0.216123,-0.304758,0.297272,0.063490,-0.155111,0.192790,0.090803,0.076976,0.226654,0.310238,-0.049298,0.095382,-0.430092,-0.348674,-0.033861,0.297530,0.052408,-0.273249,-0.018303,0.000791,-0.184625,-0.371987,-0.196453,0.265759,-0.089040,0.163914,-0.434184,0.054147,-0.180023,0.367596,0.134108,0.269924,0.013862,-0.086283,0.153577,-0.441175,-0.102963,0.169320,0.168424,-0.019549,0.066295,0.027417,-0.022250,0.115971,-0.026206,0.111903,0.134881,0.163826,0.415292,-0.045816,0.029169,-0.002534,-0.437910,-0.069225,0.127752,-0.087582,0.209030,-0.017341,-0.325876,0.031164,-0.381508,-0.209271,0.060853,-0.095384,-0.291828,-0.112984,-0.085400,-0.271536,-0.287510,-0.201039,-0.028334,0.257096,0.126726,-0.001215,0.055023,0.019923,0.145393,0.170775,0.345924,-0.001274,...,0.059508,0.037455,-0.226644,-0.200301,0.135569,-0.035710,0.130619,0.011335,-0.017847,-0.136202,0.259121,-0.040067,0.328404,-0.241552,0.123093,-0.204784,-0.206881,-0.301866,-0.098708,-0.209272,-0.543192,0.190023,0.235180,-0.119917,0.086322,0.212423,-0.129963,0.174801,0.02

In [10]:
X = cb_embeddings_df
X.shape

(1249, 384)

In [11]:
y = df["yield"]
y.shape

(1249,)

In [12]:
# Sauvegarde du DataFrame dans un fichier CSV
X.to_csv("cb_embeddings_df.csv")

In [ ]:
def filter_descriptors_low(data, threshold):
    # Drop colonne inutile
    data = data.drop(["SMILES", "Ind", "SMILES_2", "mols"], axis=1)
    # Copie des descripteurs
    descriptors = data.copy()
    
    # Calcul de la matrice de corrélation et mise à zéro de la diagonale
    corr = descriptors.corr()
    for index in corr.index:
        corr.loc[index, index] = 0
    
    # Préparation d'une liste pour collecter les noms des descripteurs non fortement corrélés
    descriptors_not_correlated = []
    
    # Vérifier chaque colonne individuellement
    for col in corr.columns:
        if all((corr[col][corr[col].notna()] <= threshold)):
            descriptors_not_correlated.append(col)
    
    print('Number of descriptors:', len(descriptors_not_correlated))
    
    # Retourner les données filtrées
    return data[descriptors_not_correlated]

In [ ]:
df_3_desc = filter_descriptors_low(df_2, 0.9)
df_3_desc.shape

In [ ]:
df_3_desc.columns

In [ ]:
import pandas as pd
import numpy as np

def filter_descriptors_hard(data, threshold):
    # Drop colonne inutile
    data = data.drop(["SMILES", "Ind", "SMILES_2", "mols"], axis=1)
    # Copie des descripteurs
    descriptors = data.copy()
    
    # Calcul de la matrice de corrélation et mise à zéro de la diagonale
    corr = descriptors.corr()
    for index in corr.index:
        corr.loc[index, index] = 0
    
    # Identifie les groupes de descripteurs fortement corrélés
    corr_mask = corr.abs() > threshold
    groups = []
    for col in corr.columns:
        correlated_with = corr.index[corr_mask[col]].tolist()
        if correlated_with and col not in sum(groups, []):
            groups.append(correlated_with)
    
    # Choix d'un descripteur par groupe
    selected_descriptors = []
    for group in groups:
        # Choix basé sur le minimum de corrélations maximales avec les autres
        min_correlation_sum = np.inf
        selected = None
        for item in group:
            # Assurez-vous de ne considérer que les valeurs non-NaN pour la somme des corrélations
            corr_sum = corr[item][corr_mask[item] & corr[item].notna()].sum()
            if corr_sum < min_correlation_sum:
                min_correlation_sum = corr_sum
                selected = item
        selected_descriptors.append(selected)
    
    print('Descripteurs sélectionnés:', selected_descriptors)
        
    # Retourner les données filtrées
    return data[selected_descriptors], data['yield']


In [ ]:
df = calculate_descriptors(df)